Classificador de sentimentos

Aplicação de algoritmo de classificador de sentimentos, sobre o dataset de reviews (em português) de filmes.

https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv

In [73]:
#importação das bibliotecas e pacotes

import pandas as pd
import nltk
import re
import string
import unicodedata

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics

In [42]:
pip install nltk==3.5

  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434693 sha256=fd747a67e887209c9840d607e1b015fcb38329712c66b5a9edf48a4453089123
  Stored in directory: c:\users\diego.plima\appdata\local\pip\cache\wheels\13\ae\bb\5e2a232ebaa1d2f38dd5f587e9fc4cf6ccb12758d14dac14d8
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7



In [43]:
import nltk

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping grammars\basqu

[nltk_data]    |   Unzipping misc\mwa_ppdb.zip.
[nltk_data]    | Downloading package names to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\names.zip.
[nltk_data]    | Downloading package nombank.1.0 to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\nonbreaking_prefixes.zip.
[nltk_data]    | Downloading package nps_chat to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\nps_chat.zip.
[nltk_data]    | Downloading package omw to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package omw-1.4 to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package opinion_lexicon to
[nltk_da

[nltk_data]    |   Unzipping taggers\universal_tagset.zip.
[nltk_data]    | Downloading package universal_treebanks_v20 to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package vader_lexicon to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package verbnet to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\verbnet.zip.
[nltk_data]    | Downloading package verbnet3 to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\verbnet3.zip.
[nltk_data]    | Downloading package webtext to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\webtext.zip.
[nltk_data]    | Downloading package wmt15_eval to
[nltk_data]    |     C:\Users\diego.plima\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping models\wmt15_eval.zip

True

In [21]:
#importando o arquivo e analisando os dados do dataset

df = pd.read_csv('reviews-pt-br.csv')
df.head()

,codigo,texto,sentimento
0,1,Esse bocejo de pia de cozinha de orçamento mui...,neg
1,2,O Bravo parece indicar que o personagem princi...,neg
2,3,"Durante a Guerra pela Independência do Sul, GE...",pos
3,4,É fora de questão que a verdadeira Anna Anders...,pos
4,5,Concordo totalmente com outro dos revisores aq...,neg


In [22]:
df.shape

(44514, 3)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44514 entries, 0 to 44513
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   codigo      44514 non-null  int64 
 1   texto       44514 non-null  object
 2   sentimento  44514 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [24]:
df['sentimento'].unique()

array(['neg', 'pos'], dtype=object)

In [44]:
#tirando a acentuação
def normalize_accents(text):
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text) #retirando pontução
    text = normalize_accents(text)
    text = re.sub(re.compile(r" +"), " ",text) #retirando espaço em branco extra
    text = re.sub(r"http\S+", "", str(text))
    return " ".join([w for w in text.split()])

#retirando pontuação
def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def tokenizer(text):
    stop_words = nltk.corpus.stopwords.words("portuguese") # portuguese, caso o dataset seja em português
    if isinstance(text, str): #verificando se o texto é uma string
        text = normalize_str(text)
        text = "".join([w for w in text if not w.isdigit()]) #remoção dos números
        text = word_tokenize(text) #tokenização
        text = [x for x in text if x not in stop_words] #retirada das stop words
        text = [y for y in text if len(y) > 2]
        return " ".join([t for t in text])
    else:
        return None

In [62]:
#aplicando o SHUFFLE
df = shuffle(df)
df = df.reset_index(drop = True)
df.head()

,texto,sentimento,texto_tratado
0,Eu vi esse filme no cinema e fiquei impression...,pos,filme cinema fiquei impressionado entao novame...
1,Desaparecimento é definido no deserto de Mojav...,neg,desaparecimento definido deserto mojave jim ha...
2,"Mesmo as pessoas que não gostam do filme, gera...",pos,pessoas nao gostam filme geralmente porque ach...
3,"Sim, definitivamente melhor do que a minha vis...",neg,sim definitivamente melhor visao death tunnel ...
4,"Eu detesto, desprezo e odeio esse filme com um...",neg,detesto desprezo odeio filme paixao faz portoe...


In [63]:
df.columns

Index(['texto', 'sentimento', 'texto_tratado'], dtype='object')

In [64]:
df = df[['texto', 'sentimento']]

In [65]:
df['texto_tratado'] = df['texto'].apply(tokenizer)

In [66]:
df.head()

,texto,sentimento,texto_tratado
0,Eu vi esse filme no cinema e fiquei impression...,pos,filme cinema fiquei impressionado entao novame...
1,Desaparecimento é definido no deserto de Mojav...,neg,desaparecimento definido deserto mojave jim ha...
2,"Mesmo as pessoas que não gostam do filme, gera...",pos,pessoas nao gostam filme geralmente porque ach...
3,"Sim, definitivamente melhor do que a minha vis...",neg,sim definitivamente melhor visao death tunnel ...
4,"Eu detesto, desprezo e odeio esse filme com um...",neg,detesto desprezo odeio filme paixao faz portoe...


In [69]:
#criando features e labels
X = df['texto_tratado'].iloc[0:99999] 
y = df['sentimento'].iloc[0:99999]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [71]:
#modelo BoW
vect = CountVectorizer(lowercase=False) 
vect.fit(X_train) #cria vocabulario
X_train = vect.transform(X_train) #criar a representação do vocabulario aprendido
X_train

<35611x108761 sparse matrix of type '<class 'numpy.int64'>'
	with 3635785 stored elements in Compressed Sparse Row format>

In [72]:
X_test = vect.transform(X_test)
X_test

<8903x108761 sparse matrix of type '<class 'numpy.int64'>'
	with 891887 stored elements in Compressed Sparse Row format>

In [74]:
clf = svm.SVC(kernel='linear') 
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [75]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8579130630124677
